In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropouta



: 

In [ ]:
# Define Quidditch team stock mapping
teams = {
    "Gryffindor": "TSLA",
    "Slytherin": "AAPL",
    "Ravenclaw": "MSFT",
    "Hufflepuff": "AMZN"
}

def fetch_stock_data(team_name, start_date="2023-01-01", end_date="2025-03-28"):
    stock_symbol = teams[team_name]
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    return stock_data[['Close']]  # We only need the closing prices

# Example usage
gryffindor_data = fetch_stock_data("Gryffindor")
gryffindor_data.tail()



In [ ]:
def preprocess_data(stock_data, sequence_length=10):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(stock_data)

    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i + sequence_length])
        y.append(scaled_data[i + sequence_length])

    X, y = np.array(X), np.array(y)
    return X, y, scaler

# Example usage
X, y, scaler = preprocess_data(gryffindor_data)


In [ ]:
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Create model
model = build_lstm_model((10, 1))


In [ ]:
# Split Data into Train and Test Sets
split = int(len(X) * 0.8)
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

# Train Model
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))


In [ ]:
# Predict
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)  # Convert back to original price scale
actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot Results
plt.figure(figsize=(12,6))
plt.plot(actual_prices, label="Actual Seeker Path (Stock Price)")
plt.plot(predictions, label="Predicted Seeker Path", linestyle="dashed")
plt.title("Seeker Trajectory Prediction (Portfolio Risk)")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.legend()
plt.show()


In [ ]:
def calculate_risk_level(actual_prices, predictions, risk_tolerance):
    # Calculate Volatility
    daily_returns = np.diff(actual_prices, axis=0) / actual_prices[:-1]
    volatility = np.std(daily_returns)

    # Set Risk Labels
    if volatility > 0.03:
        risk_level = "High Risk (Unstable Seeker)"
    elif volatility > 0.015:
        risk_level = "Medium Risk (Balanced Seeker)"
    else:
        risk_level = "Low Risk (Stable Seeker)"

    # Decision Suggestion Based on Risk Tolerance
    decision = {
        "Low": "Safe Move (Defensive Strategy)",
        "Medium": "Moderate Move (Balanced Strategy)",
        "High": "Risky Move (Aggressive Trading)"
    }[risk_tolerance]

    return risk_level, volatility, decision

# Example Usage
risk_level, volatility, decision = calculate_risk_level(actual_prices, predictions, "Medium")
print(f"Risk Level: {risk_level}")
print(f"Volatility Score: {volatility:.4f}")
print(f"Suggested Decision: {decision}")


In [ ]:
# Visualization
plt.figure(figsize=(10, 5))
for idx, team in enumerate(teams.keys()):
    plt.plot(stock_df.index[-30:], stock_df[team][-30:], label=f"{team} Actual", linestyle='dotted')
    plt.scatter(stock_df.index[-1], predicted_prices[0][idx], label=f"{team} Prediction", marker='x')
plt.legend()
plt.title("Seeker Trajectory Prediction (Stock Trends)")
plt.xlabel("Date")
plt.ylabel("Stock Price ($)")
plt.show()
print("LSTM Model trained and predictions visualized.")

In [ ]:
model.save("lstm_stock_model.h5")


In [ ]:
model = load_model("lstm_stock_model.h5")

In [ ]:


# Mapping Quidditch Teams to Stock Symbols
teams = {
    "Gryffindor": "TSLA",
    "Slytherin": "AAPL",
    "Ravenclaw": "MSFT",
    "Hufflepuff": "AMZN"
}

# User Input
team_name = input("Enter Quidditch Team (Gryffindor/Slytherin/Ravenclaw/Hufflepuff): ")
risk_tolerance = input("Enter Risk Tolerance (Low/Medium/High): ")
time_frame = input("Enter Time Frame (Short-Term/Medium-Term/Long-Term): ")

# Fetch Stock Data
stock_symbol = teams.get(team_name, "TSLA")  # Default to TSLA if invalid
data = yf.download(stock_symbol, period="2y")  # Fetch last 2 years of data
prices = data[['Close']]

# Scale Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_prices = scaler.fit_transform(prices)

# Get Last 10 Days for Prediction
X_test = np.array([scaled_prices[-10:]])  # Last 10 days data as input

# Predict
predicted_price = model.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price)[0][0]  # Convert back to actual price

# Output Prediction
print(f"\n Predicted Seeker Path (Stock Price) for {team_name}: ${predicted_price:.2f}")

# Risk Analysis Logic
if risk_tolerance.lower() == "low":
    decision = "Safe Move (Defensive Strategy)"
elif risk_tolerance.lower() == "medium":
    decision = "Moderate Move (Balanced Strategy)"
else:
    decision = "Risky Move (Aggressive Trading)"

print(f"Suggested Strategy: {decision}")
